### 层次分析法

In [1]:
import numpy as np


def calculate_CR_W(M: np.ndarray):
    """
    计算CR值

    Params:
    ---
    M: np.ndarray
        对比矩阵

    Returns:
    ---
    CR: float
        CR值，一般小于 0.1 通过检验
    W: np.ndarray
        权重向量
    """
    n = M.shape[1]  # 评价指标个数
    RI = [0, 0, 0.58, 0.90, 1.12, 1.24, 1.32, 1.41, 1.45, 1.49, 1.51]
    # 求判断矩阵的特征值和特征向量，V为特征值，D为特征向量
    V, D = np.linalg.eig(M)  # 注意这里的结果是复数，有虚部
    # 求矩阵的最大特征值
    # eig 方法已经排好了，所以直接取第一个即可
    v, d = V[0], D[0]
    W = d / np.sum(d)  # 归一化后的权重向量
    CI = (v - n) / (n - 1)
    CR = CI / RI[n]
    return CR, W

In [2]:
# 构建矩阵
M = np.array(
    [
        [1, 1 / 5, 1 / 3, 1],
        [5, 1, 3, 5],
        [3, 1 / 3, 1, 3],
        [1, 1 / 5, 1 / 3, 1],
    ]
)

CR, W = calculate_CR_W(M)
if CR<0.10:
    print("CR=",CR)
    print("对比矩阵A通过一致性检验，各向量权重向量Q为：")
    print(W)
else:
    print("对比矩阵A未通过一致性检验，需对对比矩阵A重新构造")

CR= (0.012944482621398318+0j)
对比矩阵A通过一致性检验，各向量权重向量Q为：
[-0.22963642+0.j          0.08125132+0.10775945j  0.08125132-0.10775945j
  1.06713377+0.j        ]
